In [5]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
import openai
import os




In [17]:
template = """Answer the users question based only on the following context:

<context>
{context}
</context>

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(temperature=0,api_key="")

search = DuckDuckGoSearchAPIWrapper()


def retriever(query):
    return search.run(query)

In [12]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [13]:
simple_query = "what is langchain?"

In [14]:
chain.invoke(simple_query)

'LangChain is a powerful tool for building LLM-powered applications, simplifying AI application development by integrating various language models and tools. It offers features for data communication, vector embeddings, and interaction with LLMs, enabling more contextually aware AI applications.'

In [15]:
retriever(simple_query)

"LangChain is an open-source framework designed to simplify the creation of applications using large language models (LLMs). It provides a standard interface for chains, lots of integrations with other tools, and end-to-end chains for common applications. LangChain has a set_debug() method that will return more granular logs of the chain internals: Let's see it with the above example. First, we'll need to install the main langchain package for the entrypoint to import the method: %pip install langchain. Then add this code: from langchain.globals import set_debug. LangChain simplifies AI application development by integrating various language models and tools. It offers features for data communication, vector embeddings, and interaction with LLMs, enabling more contextually aware AI applications. LangChain is a framework for working with large language models in Java. In this article, you will learn how to use LangChain to perform tasks such as text generation, summarization, translatio

In [16]:
template = """Provide a better search query for \
web search engine to answer the given question, end \
the queries with ’**’. Question: \
{x} Answer:"""
rewrite_prompt = ChatPromptTemplate.from_template(template)

In [19]:
from langchain import hub

rewrite_prompt = hub.pull("langchain-ai/rewrite")

In [20]:
print(rewrite_prompt.template)

Provide a better search query for web search engine to answer the given question, end the queries with ’**’.  Question {x} Answer:


In [21]:
# Parser to remove the `**`


def _parse(text):
    return text.strip('"').strip("**")

In [23]:
rewriter = rewrite_prompt | model | StrOutputParser() | _parse

In [24]:
rewriter.invoke({"x": simple_query})

'What is Langchain technology and how does it work?'

In [ ]:
rewrite_retrieve_read_chain = (
    {
        "context": {"x": RunnablePassthrough()} | rewriter | retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [13]:
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine.transform_query_engine import (
    TransformQueryEngine,
)
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [16]:
# Initialize the gpt3.5 model
gpt3 = OpenAI( api_key=OPENAI_API_KEY)
# Initialize the embedding model
embed_model = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

service_context_gpt3 = ServiceContext.from_defaults(llm=gpt3, chunk_size = 256, chunk_overlap=0, embed_model=embed_model)



C:\Users\pduy3\AppData\Local\Temp\ipykernel_21500\3190974141.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context_gpt3 = ServiceContext.from_defaults(llm=gpt3, chunk_size = 256, chunk_overlap=0, embed_model=embed_model)


AttributeError: 'OpenAI' object has no attribute 'system_prompt'

In [ ]:
# build index and query engine for the index
index = VectorStoreIndex.from_documents(documents, service_context=service_context_gpt3)
query_engine = index.as_query_engine(similarity_top_k=4)

# HyDE setup
hyde = HyDEQueryTransform(include_original=True)
# Transform the query engine using HyDE
hyde_query_engine = TransformQueryEngine(query_engine, hyde)

# Print the response from the model
response = hyde_query_engine.query("Compare the families of Emma Stone and Ryan Gosling")
print(response)

In [12]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from langchain import hub
from llama_index.core import (VectorStoreIndex, download_loader)
from llama_index import ServiceContext


import os
from dotenv import load_dotenv

load_dotenv()


ImportError: cannot import name 'ServiceContext' from 'llama_index' (unknown location)

In [19]:
import sys 
import os

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [24]:
documents = SimpleDirectoryReader(
      input_files=["temp.pdf"]
    ).load_data()
index = VectorStoreIndex.from_documents(documents)

TypeError: 'VectorStoreIndex' object is not subscriptable